In [1]:
import numpy as np
import pandas as pd
from typing import List, Dict

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
songs = pd.read_csv('songdata.csv')
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [5]:
songs['text'] = songs['text'].str.replace(r'\n', '')

In [6]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [7]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

In [8]:
similarities = {}
for i in range(len(cosine_similarities)):
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [9]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        song = recommendation['song']
        number_songs = recommendation['number_songs']
        recom_song = self.matrix_similar[song][:number_songs]
        self._print_message(song=song, recom_song=recom_song)

In [10]:
recommedations = ContentBasedRecommender(similarities)
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}
recommedations.recommend(recommendation)

The 4 recommended songs for Beneath Still Waters are:
Number 1:
Beneath Still Waters by Emmylou Harris with 0.98 similarity score
--------------------
Number 2:
It's Too Soon To Know by Linda Ronstadt with 0.238 similarity score
--------------------
Number 3:
Gone Too Long by Gloria Gaynor with 0.221 similarity score
--------------------
Number 4:
Gone by Diana Ross with 0.212 similarity score
--------------------


In [11]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}
recommedations.recommend(recommendation2)

The 4 recommended songs for Feel Good are:
Number 1:
You Give Good Love by Whitney Houston with 0.51 similarity score
--------------------
Number 2:
Good Enough by Evanescence with 0.498 similarity score
--------------------
Number 3:
Roulette by System Of A Down with 0.46 similarity score
--------------------
Number 4:
I Feel So Good by Nick Cave with 0.446 similarity score
--------------------
